In [1]:
import numpy as np
import pandas as pd 
import pickle
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
'''
train_1 = pd.read_csv('D:\\xgb_expmean_1_train.csv')
train_2 = pd.read_csv('D:\\xgb_expmean_2_train.csv')
train_3 = pd.read_csv('D:\\xgb_expmean_3_train.csv')
train_6 = pd.read_csv('D:\\xgb_expmean_6_train.csv')
train_9 = pd.read_csv('D:\\xgb_expmean_9_train.csv')

valid_1 = pd.read_csv('D:\\xgb_expmean_1_valid.csv')
valid_2 = pd.read_csv('D:\\xgb_expmean_2_valid.csv')
valid_3 = pd.read_csv('D:\\xgb_expmean_3_valid.csv')
valid_6 = pd.read_csv('D:\\xgb_expmean_6_valid.csv')
valid_9 = pd.read_csv('D:\\xgb_expmean_9_valid.csv')
'''

In [2]:
NUM_MODELS=3

In [3]:
train_1 = pd.read_csv('xgb_date_1_3_6_9_12_train.csv')
train_2 = pd.read_csv('xgb_date_2_4_5_7_8_train.csv')
train_3 = pd.read_csv('xgb_expmean_train.csv')

valid_1 = pd.read_csv('xgb_date_1_3_6_9_12_valid.csv') 
valid_2 = pd.read_csv('xgb_date_2_4_5_7_8_valid.csv')
valid_3 = pd.read_csv('xgb_expmean_valid.csv')

In [4]:
train_1.head()

,Unnamed: 0,pred_targets,targets
0,4488756,2.044595,4.0
1,4488757,0.922610,3.0
2,4488758,12.097198,14.0
3,4488759,1.587751,3.0
4,4488760,0.266861,1.0


In [5]:
train_2.head()

,pred_targets,targets
0,0.421632,4.0
1,1.798981,3.0
2,13.013217,14.0
3,4.435820,3.0
4,0.258385,1.0


In [6]:
# Predicted targets will be training data
X_train_1 = train_1['pred_targets']
X_train_2 = train_2['pred_targets']
X_train_3 = train_3['pred_targets']
#X_train_6 = train_6['pred_targets']
#X_train_9 = train_9['pred_targets']

In [7]:
len(X_train_1)

6186922

In [8]:
len(X_train_2)

6186922

In [9]:
X_train = pd.concat([X_train_1,X_train_2,X_train_3],axis=1)
X_train.columns = range(NUM_MODELS)
X_train.head()

,0,1,2
0,2.044595,0.421632,1.279308
1,0.922610,1.798981,0.693778
2,12.097198,13.013217,13.878383
3,1.587751,4.435820,2.370638
4,0.266861,0.258385,0.299256


In [10]:
X_train.tail()

,0,1,2
6186917,0.237601,0.132207,0.157526
6186918,0.252929,0.076806,0.124611
6186919,0.055970,0.111051,0.135284
6186920,0.117676,0.107635,0.124611
6186921,0.234482,0.107635,0.124611


In [11]:
# Y_train will be original target vales
Y_train = train_1['targets']
Y_train.head()

0     4.0
1     3.0
2    14.0
3     3.0
4     1.0
Name: targets, dtype: float64

In [12]:
X_valid_1 = valid_1['pred_targets']
X_valid_2 = valid_2['pred_targets']
X_valid_3 = valid_3['pred_targets']
#X_valid_6 = valid_6['pred_targets']
#X_valid_9 = valid_9['pred_targets']

X_valid = pd.concat([X_valid_1,X_valid_2,X_valid_3],axis=1)
X_valid.columns = range(NUM_MODELS)
X_valid.head()

,0,1,2
0,0.049207,0.058917,0.046695
1,0.625949,1.055997,0.811740
2,0.404079,0.436859,0.642842
3,0.123963,0.170124,0.073209
4,0.983192,0.769772,0.493259


In [13]:
# Y_valid will be original target vales
Y_valid = valid_1['targets']
Y_valid.head()

0    1.0
1    1.0
2    2.0
3    1.0
4    1.0
Name: targets, dtype: float64

In [14]:
df = X_train
missing_values_per_column = df.isnull().sum(axis=0,skipna=False)
missing_values_per_column /= df.shape[0]
missing_values_per_column = missing_values_per_column.tolist()
df_missing_col = pd.Series(missing_values_per_column, index=df.columns.values)
print("Missing Values per Column For X_train")
print(df_missing_col)

Missing Values per Column For X_train
0    0.0
1    0.0
2    0.0
dtype: float64


In [19]:
model = Ridge(alpha=0.1, copy_X=True, normalize=True, max_iter=1000,random_state=3)
model.fit(X_train, Y_train)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, random_state=3, solver='auto', tol=0.001)

In [21]:
# Save model
pickle.dump(model, open('ensemble_model.pickle.dat', "wb"))

In [20]:
train_pred = model.predict(X_train)
valid_pred = model.predict(X_valid)
train_rmse = np.sqrt(mean_squared_error(Y_train, train_pred))
valid_rmse = np.sqrt(mean_squared_error(Y_valid, valid_pred))
print("Train RMSE: "+str(train_rmse))
print("Valid RMSE: "+str(valid_rmse))

Train RMSE: 0.7913524811080762
Valid RMSE: 0.8874467956341665


Ensemble of xgb_date_1_3_6_9_12, xgb_date_2_4_5_7_8 and xgb_expmean

alpha=0.1 ... Train RMSE: 0.7913524811080762 ... Valid RMSE: 0.8874467956341665 ... Test RMSE: 0.92212

alpha=0.2 ... Train RMSE: 0.7936686104072199 ... Valid RMSE: 0.8875424778628641

In [ ]:
# load model
model = pickle.load(open('ensemble_model.pickle.dat', "rb"))

In [22]:
# Read test data
test_1 = pd.read_csv('xgb_date_1_3_6_9_12_submission.csv') 
test_2 = pd.read_csv('xgb_date_2_4_5_7_8_submission.csv') 
test_3 = pd.read_csv('xgb_expmean_submission.csv')
#test_6 = pd.read_csv('xgb_expmean_6_submission.csv')
#test_9 = pd.read_csv('xgb_expmean_9_submission.csv')

In [23]:
test_1.head()

,ID,item_cnt_month
0,0,0.560161
1,1,0.399338
2,2,0.945301
3,3,0.524785
4,4,3.484849


In [24]:
test_1.tail()

,ID,item_cnt_month
214195,214195,0.087229
214196,214196,0.133401
214197,214197,0.070579
214198,214198,0.034684
214199,214199,0.045824


In [25]:
X_test_1 = test_1['item_cnt_month']
X_test_2 = test_2['item_cnt_month']
X_test_3 = test_3['item_cnt_month']
#X_test_6 = test_6['item_cnt_month']
#X_test_9 = test_9['item_cnt_month']

X_test = pd.concat([X_test_1,X_test_2,X_test_3],axis=1)
X_test.columns = range(NUM_MODELS)
X_test.head()

,0,1,2
0,0.560161,1.094839,0.559474
1,0.399338,0.436963,0.098269
2,0.945301,1.302916,0.930836
3,0.524785,0.468809,0.251569
4,3.484849,1.995806,1.064806


In [26]:
test_pred = model.predict(X_test)
test_pred

array([0.77397537, 0.33356236, 1.1392437 , ..., 0.04345263, 0.0183347 ,
       0.02700878])

In [27]:
X_test['ID'] = X_test.index
X_test['item_cnt_month'] = test_pred.clip(0, 20)
X_test[['ID','item_cnt_month']].to_csv('final_submission.csv',index=False)